# Proportional Score Voting

Also called Reweighted Range Voting

In [56]:
import pandas

ballots = [
  {"Red": 5, "Green": 0, "Yellow": 3, "Blue": 5},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
  {"Red": 0, "Green": 5, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 2, "Yellow": 4, "Blue": 3}, 
  {"Red": 1, "Green": 0, "Yellow": 2, "Blue": 0},  
  {"Red": 1, "Green": 3, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 0, "Yellow": 5, "Blue": 0},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
]

seats = 4
seated = []
max_score = max(max(ballot.values()) for ballot in ballots)

def reweight(ballot):
  seated_scores = [
      ballot[candidate] for candidate in ballot if candidate in seated
  ]
  weight = 1/(1+sum(seated_scores)/max_score)
  return {candidate: weight*ballot[candidate] for candidate in ballot}

def nextRound(ballots):
  reweightedBallots = [reweight(ballot) for ballot in ballots] 
  winner = pandas.DataFrame(reweightedBallots).sum().drop(seated).idxmax()
  print(pandas.DataFrame(reweightedBallots).sum())
  seated.append(winner)
  return reweightedBallots

while len(seated) < seats:
  nextRound(ballots)
  print(seated)


Red       18.0
Green     10.0
Yellow    14.0
Blue      18.0
dtype: float64
['Red']
Red       10.000000
Green      9.166667
Yellow    11.500000
Blue      10.833333
dtype: float64
['Red', 'Yellow']
Red       8.881410
Green     8.500000
Yellow    6.903846
Blue      9.256410
dtype: float64
['Red', 'Yellow', 'Blue']
Red       6.684219
Green     7.078755
Yellow    6.121795
Blue      6.947497
dtype: float64
['Red', 'Yellow', 'Blue', 'Green']


# Sequential Proportional Approval Voting

This is functionally equivalent to Proportional Score Voting, just max_score is 1.

In [57]:
import pandas

ballots = [
    {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 0},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
    {"Red": 1, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
    {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
]

SEATS = 4

seated = []
max_score = max(max(ballot.values())
                for ballot in ballots)  # 1 for Approval Voting


def reweight(ballot):
    seated_scores = [
        ballot[candidate] for candidate in ballot if candidate in seated
    ]
    weight = 1/(1+sum(seated_scores)/max_score)
    return {candidate: weight*ballot[candidate] for candidate in ballot}


def nextRound(ballots):
    reweighted_ballots = [reweight(ballot) for ballot in ballots]
    winner = pandas.DataFrame(reweighted_ballots).sum().drop(seated).idxmax()
    print(pandas.DataFrame(reweighted_ballots).sum())
    seated.append(winner)
    return reweighted_ballots


while len(seated) < SEATS:
    nextRound(ballots)
    print(f"Winners: {seated}")


Red       6.0
Green     4.0
Yellow    3.0
Blue      8.0
dtype: float64
Winners: ['Blue']
Red       3.5
Green     2.0
Yellow    2.0
Blue      4.0
dtype: float64
Winners: ['Blue', 'Red']
Red       2.166667
Green     1.833333
Yellow    1.166667
Blue      3.166667
dtype: float64
Winners: ['Blue', 'Red', 'Green']
Red       2.083333
Green     1.250000
Yellow    1.166667
Blue      2.583333
dtype: float64
Winners: ['Blue', 'Red', 'Green', 'Yellow']


In [58]:
import pandas

ballots = [
  {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 0}, 
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 1, "Blue": 1},  
  {"Red": 1, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 1, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 0, "Yellow": 0, "Blue": 1},
]
print(pandas.DataFrame(ballots).sum())
print(f"{pandas.DataFrame(ballots).sum().idxmax()} wins")


Red       6
Green     4
Yellow    3
Blue      8
dtype: int64
Blue wins


In [59]:
# Instant Runoff Voting
import pandas

# Ranking a scored ballot because ranked ballots are bad, and annoying to compose.
ballots = [
  {"Red": 5, "Green": 0, "Yellow": 3, "Blue": 4},
  {"Red": 5, "Green": 0, "Yellow": 1, "Blue": 4},
  {"Red": 0, "Green": 5, "Yellow": 0, "Blue": 1},
  {"Red": 1, "Green": 2, "Yellow": 4, "Blue": 3}, 
  {"Red": 1, "Green": 0, "Yellow": 2, "Blue": 0},  
  {"Red": 1, "Green": 3, "Yellow": 0, "Blue": 1},
  {"Red": 0, "Green": 0, "Yellow": 5, "Blue": 0},
  {"Red": 5, "Green": 0, "Yellow": 0, "Blue": 4},
]

ballots = pandas.DataFrame(ballots)

def nextRound(ballots):
    # get highest rated candidate from each row, count the number of times it occurs
    round_count = pandas.DataFrame(ballots).apply(lambda x: x.idxmax(), axis=1).value_counts()
    print(round_count)
    # find the lowest count
    eliminated = round_count.idxmin()
    print(f"{eliminated =}")
    # remove the eliminated candidates from the ballots
    ballots.drop(eliminated, axis=1, inplace=True)
    return ballots

while len(ballots.columns) > 1:
    nextRound(ballots)
f'Winner: {ballots.columns[0]}'


Red       3
Yellow    3
Green     2
dtype: int64
eliminated ='Green'
Red       4
Yellow    3
Blue      1
dtype: int64
eliminated ='Blue'
Red       5
Yellow    3
dtype: int64
eliminated ='Yellow'


'Winner: Red'

In [72]:
# Single Transferrable Vote
import pandas
import numpy

# Ranking a scored ballot because ranked ballots are bad, and annoying to compose.
ballots = [

    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},
    {"A": 5, "B": 4, "C": 3, "D": 0, "E": 0, "F": 0},

    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},
    {"A": 0, "B": 5, "C": 0, "D": 4, "E": 2, "F": 1},

    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},
    {"A": 4, "B": 0, "C": 1, "D": 0, "E": 0, "F": 5},

    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},
    {"A": 0, "B": 0, "C": 4, "D": 5, "E": 3, "F": 2},

    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0},
    {"A": 0, "B": 3, "C": 5, "D": 4, "E": 0, "F": 0}

]

seats = 3
ballots = pandas.DataFrame(ballots)
quota = 1+numpy.floor(len(ballots) / (seats + 1))
seated = []

weights = pandas.Series(numpy.ones(len(ballots)), name='weight')


def choices(ballots):
    return pandas.DataFrame(ballots.apply(lambda x: x.idxmax(), axis=1), columns=['choice'])


def summarize(ballots):
    # count highest rated candidate from each row, then count the number of times it occurs
    return pandas.concat([choices(ballots), weights], axis=1).groupby('choice').sum()


def eliminate_zeros(ballots):
    # eliminate candidates who have no first place votes
    eliminated = ballots.columns.difference(summarize(ballots).index)
    if not eliminated.empty: 
        print(f"{eliminated[0] = }, no first place votes")
    return ballots.drop(eliminated, axis=1)


def eliminate_lowest(ballots):
    # if no candidate meets the quota, eliminate the worst performing candidate
    round_count = summarize(ballots)
    if round_count[round_count['weight'] >= quota].empty:
        eliminated = round_count.idxmin().values[0]
        print(f"{eliminated[0] = }, no candidate meets quota, eliminated lowest")
        return ballots.drop(eliminated, axis=1)
    return ballots


def elect_and_distribute_excess(ballots, weights):
    round_count = summarize(ballots)
    # Elect the candidates who have passed the quota
    winner = round_count[round_count['weight'] >= quota].index[0]
    seated.extend(winner)
    excess_votes = round_count.loc[winner].values[0] - quota
    print(f"{seated =}, {quota = }, {excess_votes = }")
    fractional_votes = excess_votes / round_count.max()
    # divide the excess votes by the number of votes for the winner for the value of the excess votes

    # find the second choice votes of those who voted for the winner
    voted_for_winner = choices(ballots)[choices(ballots) == winner].dropna()
    ballots = ballots.drop(winner, axis=1)
    # redistribute the excess votes to the second choices
    weights.iloc[voted_for_winner.index] = fractional_votes
    return ballots, weights


while len(seated) < seats:
    ballots = eliminate_zeros(ballots)
    ballots = eliminate_lowest(ballots)
    ballots, weights = elect_and_distribute_excess(ballots, weights)


eliminated[0] = 'E', no first place votes
eliminated[0] = 'C', no candidate meets quota, eliminated lowest
seated =['D'], quota = 25.0, excess_votes = 14.0
seated =['D', 'B'], quota = 25.0, excess_votes = 1.8205128205128212
seated =['D', 'B', 'F'], quota = 25.0, excess_votes = 3.519109820996615
